## 📈 시계열 데이터 분석 (Part 1)
### 1. 시계열 데이터(Time Series Data)의 중요성
### 💡 개념 (Concept)
> **시계열 데이터**는 일정 시간 간격으로 기록된 데이터의 연속입니다. 주가, 날씨, 웹사이트 트래픽, 판매량 등 우리 주변의 수많은 데이터가 시계열 데이터에 해당합니다.
>
> 이러한 데이터를 올바르게 분석하고 미래를 예측하는 능력은 비즈니스 의사결정, 리소스 최적화, 과학적 연구 등 다양한 분야에서 핵심적인 역량으로 자리 잡았습니다.
>
> **Pandas**는 파이썬에서 시계열 데이터를 다루는 데 가장 강력하고 필수적인 라이브러리입니다. 날짜/시간 데이터를 쉽게 생성, 조작, 분석할 수 있는 풍부한 기능을 제공하여 시계열 분석의 전 과정을 효율적으로 만듭니다.

### 💭 이 파트에서 배울 것

> - Pandas를 사용하여 날짜 및 시간 데이터를 생성하고 변환하는 방법
> - 시계열 데이터의 주기를 변경하고 집계하는 리샘플링(Resampling)
> - 이동 평균(Moving Average)과 같은 이동창(Window) 함수 활용법
> - 데이터의 시점을 이동시키는 시프트(Shift)와 차분(Diff)
> - 실제 데이터를 활용한 미니 프로젝트 실습

--- 

## 2. 시계열 데이터 생성 및 변환

### 💡 개념 (Concept)

분석을 시작하려면 먼저 데이터를 '시간'의 속성을 가진 객체로 만들어야 합니다. Pandas는 문자열로 된 날짜를 `datetime` 객체로 변환하거나, 특정 기간의 날짜를 생성하는 등 다양한 방법을 제공합니다.

### `pd.to_datetime`: 문자열을 날짜 객체로
> 텍스트 파일이나 데이터베이스에서 가져온 날짜 데이터는 종종 문자열(object) 타입입니다. `pd.to_datetime` 함수는 이러한 문자열을 Pandas가 인식할 수 있는 `datetime` 객체로 변환하여 시간 기반의 계산과 분석을 가능하게 합니다.

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

# 예제 데이터프레임 생성
date_strings = ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05']
values = [10, 15, 12, 18, 20]
df_str = pd.DataFrame({'date_str': date_strings, 'value': values})

In [10]:

print("--- 변환 전 --- ")
print(df_str.info())

# 'date_str' 컬럼을 datetime 객체로 변환
df_str['date'] = pd.to_datetime(df_str['date_str'])

print("--- 변환 후 ---")
print(df_str.info())

df_str.head()

--- 변환 전 --- 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date_str  5 non-null      object
 1   value     5 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes
None
--- 변환 후 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date_str  5 non-null      object        
 1   value     5 non-null      int64         
 2   date      5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 252.0+ bytes
None


,date_str,value,date
0,2023-01-01,10,2023-01-01
1,2023-01-02,15,2023-01-02
2,2023-01-03,12,2023-01-03
3,2023-01-04,18,2023-01-04
4,2023-01-05,20,2023-01-05


### ✏️ 연습 문제 (Practice Problems)

1. 다음은 여러 국가의 날짜 형식이 섞여있는 리스트입니다. `pd.to_datetime`을 사용하여 모두 표준 `datetime` 객체로 변환하고, 변환된 데이터의 타입을 확인하세요.
   - `date_list = ['07-15-2023', '2023/08/20', '2023.09.25']`

In [11]:
from dateutil import parser

In [12]:
parser.parse('07-15-2023')

datetime.datetime(2023, 7, 15, 0, 0)

In [16]:
# 시계열 문자 전처리
# 방법 1
date_list = ['07-15-2023', '2023/08/20', '2023.09.25']
# pd.Series(list(map(parser.parse, date_list)))

# 방법2
datetime_series = pd.Series(date_list).apply(parser.parse)

0   2023-07-15
1   2023-08-20
2   2023-09-25
dtype: datetime64[ns]

In [ ]:
# 연습 문제 1번 풀이 공간
date_list = ['07-15-2023', '2023/08/20', '2023.09.25']
datetime_series = pd.to_datetime(date_list)
print(datetime_series)
print("Data Type:", datetime_series.dtype)

### `pd.date_range`: 특정 기간의 날짜 생성
> `pd.date_range` 함수를 사용하면 시작일, 종료일, 그리고 빈도(frequency)를 지정하여 손쉽게 날짜/시간 인덱스를 만들 수 있습니다. 이는 데이터가 없는 기간을 채우거나 분석의 기준이 되는 시간을 생성할 때 매우 유용합니다.

| freq 코드 | 설명 |
|---|---|
| 'D' | 일(Day) |
| 'W' | 주(Week) |
| 'M' | 월말(Month End) |
| 'MS'| 월초(Month Start)|
| 'Q' | 분기말(Quarter End) |
| 'A' | 연말(Year End) |
| 'H' | 시(Hour) |
| 'T' | 분(Minute) |
| 'S' | 초(Second) |

In [4]:
# 2023년 1월 한 달간의 날짜 인덱스 생성 (빈도: 'D' - 일)
dates_2023_jan = pd.date_range(start='2023-01-01', end='2023-01-31', freq='D')
print(dates_2023_jan)

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-14', '2023-01-15', '2023-01-16',
               '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20',
               '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24',
               '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28',
               '2023-01-29', '2023-01-30', '2023-01-31'],
              dtype='datetime64[ns]', freq='D')


### ✏️ 연습 문제 (Practice Problems)

1. `pd.date_range`를 사용하여 2024년 6월의 모든 평일(월-금) 날짜를 생성해보세요. (`freq` 옵션을 찾아보세요! 힌트: 'Business day')

In [17]:
# 연습 문제 1번 풀이 공간
weekdays_in_june = pd.date_range(start='2024-06-01', end='2024-06-30', freq='B')
print(weekdays_in_june)

DatetimeIndex(['2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06',
               '2024-06-07', '2024-06-10', '2024-06-11', '2024-06-12',
               '2024-06-13', '2024-06-14', '2024-06-17', '2024-06-18',
               '2024-06-19', '2024-06-20', '2024-06-21', '2024-06-24',
               '2024-06-25', '2024-06-26', '2024-06-27', '2024-06-28'],
              dtype='datetime64[ns]', freq='B')


--- 

## 3. 시계열 데이터 인덱싱과 리샘플링

### 💡 개념 (Concept)

> 시계열 데이터의 핵심은 '시간'을 기준으로 데이터를 자르고, 변형하고, 집계하는 것입니다. 날짜를 데이터프레임의 인덱스(index)로 설정하면 강력한 시계열 분석 기능을 활용할 수 있습니다. 
> 
> **리샘플링(Resampling)** 은 시계열 데이터의 시간 빈도를 변경하는 과정입니다. 
> 
> 예를 들어, 일별(daily) 데이터를 주별(weekly)이나 월별(monthly) 데이터로 변환하여 큰 흐름을 보거나(다운샘플링), 분 단위 데이터를 초 단위로 세분화(업샘플링)할 수 있습니다.

### `set_index`와 `resample`: 시간 단위로 데이터 집계하기
> `set_index`로 날짜 컬럼을 인덱스로 지정한 후, `resample()` 메소드를 사용하여 원하는 시간 단위로 데이터를 그룹화하고, `.mean()`, `.sum()`, `.last()` 등과 같은 집계 함수를 적용할 수 있습니다.

In [18]:
# 실습을 위한 예제 데이터 생성
# 날짜 인덱스를 잡고 사용
date_rng = pd.date_range(start='2023-01-01', periods=100, freq='D')
np.random.seed(42)
data = np.random.randint(50, 200, size=(100,))
ts_df = pd.DataFrame(data, index=date_rng, columns=['sales'])

print("--- 원본 데이터 (일별 판매량) ---")
print(ts_df.head())

# 주별(Weekly) 평균 판매량 계산 (다운샘플링)
# 'W'는 주의 마지막 날(일요일)을 기준으로 집계합니다.
weekly_mean_sales = ts_df['sales'].resample('W').mean()

print("--- 주별 평균 판매량 ---")
print(weekly_mean_sales.head())

--- 원본 데이터 (일별 판매량) ---
            sales
2023-01-01    152
2023-01-02    142
2023-01-03     64
2023-01-04    156
2023-01-05    121
--- 주별 평균 판매량 ---
2023-01-01    152.000000
2023-01-08    125.142857
2023-01-15    158.285714
2023-01-22    104.714286
2023-01-29    105.142857
Freq: W-SUN, Name: sales, dtype: float64


In [19]:
# 월별(Monthly) 총 판매량 계산 (다운샘플링)
# 'MS'는 월의 시작일(1일)을 기준으로 집계합니다.
monthly_sum_sales = ts_df['sales'].resample('MS').sum()

print("--- 월별 총 판매량 ---")
print(monthly_sum_sales.head())

# Plotly를 사용한 시각화
fig = px.bar(monthly_sum_sales, 
             title='월별 총 판매량', 
             labels={'value':'총 판매량', 'index':'월'})
fig.show()

--- 월별 총 판매량 ---
2023-01-01    3822
2023-02-01    3375
2023-03-01    3678
2023-04-01    1120
Freq: MS, Name: sales, dtype: int32


In [21]:
import plotly

In [31]:
# plotly 로 pandas 바로 시각화
pd.options.plotting.backend = "plotly"
monthly_sum_sales.bar().show()

AttributeError: 'Series' object has no attribute 'bar'

### ✏️ 연습 문제 (Practice Problems)

1. 위에서 생성한 `ts_df` 데이터를 사용하여, **분기별(Quarterly) 판매량의 최댓값**을 계산하고 출력해보세요. (`freq` 코드는 'Q'를 사용합니다.)

In [23]:
# 연습 문제 1번 풀이 공간
quarterly_max_sales = ts_df['sales'].resample('Q').max()
print(quarterly_max_sales)

2023-03-31    199
2023-06-30    184
Freq: QE-DEC, Name: sales, dtype: int32


--- 

## 4. 이동창 함수 (Window Functions)

### 💡 개념 (Concept)

> 시계열 데이터의 노이즈(noise)를 줄이고 장기적인 추세를 파악하기 위해 **이동창(Window)** 함수를 사용합니다. 이는 일정 크기의 창(window)을 데이터 위에서 한 칸씩 이동시키면서 창 내부의 데이터로 통계를 계산하는 방식입니다.
> 
> - **`rolling()`**: 고정된 크기의 창을 사용합니다. 예를 들어 '7일 이동 평균'은 최근 7일간의 데이터 평균을 계속 계산하여 데이터의 단기 변동성을 완화하고 추세를 부드럽게 보여줍니다.
> - **`expanding()`**: 창의 크기가 데이터가 추가될수록 계속 커집니다. 시작점부터 현재 지점까지의 모든 데이터를 사용하여 누적 합계, 누적 평균 등을 계산할 때 사용됩니다.

### `rolling`: 이동 평균으로 추세 파악하기
> `rolling(window=크기)`으로 이동창 객체를 만들고, `.mean()`, `.std()` 등 집계 함수를 적용합니다. `window`는 계산에 포함할 데이터의 개수를 의미합니다.

In [24]:
# 위에서 사용한 ts_df 데이터를 계속 사용합니다.

# 7일 이동 평균 계산
ts_df['sales_MA_7'] = ts_df['sales'].rolling(window=7).mean()

# 30일 이동 평균 계산
ts_df['sales_MA_30'] = ts_df['sales'].rolling(window=30).mean()

print(ts_df.head(10)) # 처음 6개는 NaN값이 나옵니다.

# Plotly를 사용하여 원본 데이터와 이동 평균선 시각화
fig = px.line(ts_df, y=['sales', 'sales_MA_7', 'sales_MA_30'], title='일별 판매량과 이동 평균')
fig.show()

            sales  sales_MA_7  sales_MA_30
2023-01-01    152         NaN          NaN
2023-01-02    142         NaN          NaN
2023-01-03     64         NaN          NaN
2023-01-04    156         NaN          NaN
2023-01-05    121         NaN          NaN
2023-01-06     70         NaN          NaN
2023-01-07    152  122.428571          NaN
2023-01-08    171  125.142857          NaN
2023-01-09    124  122.571429          NaN
2023-01-10    137  133.000000          NaN


### `rolling.apply`:
 > `rolling(window=크기)`로 이동창 객체를 만든 뒤, `.apply(사용자정의함수)`를 사용하면 창(window) 내의 데이터에 대해 직접 정의한 함수를 적용할 수 있습니다.
 > 예를 들어, 이동창 내에서 중앙값, 최댓값, 또는 복잡한 계산을 하고 싶을 때 활용합니다.


In [27]:
ts_df['sales_Median_7'] = ts_df['sales'].rolling(window=7).apply(np.median)

### ✏️ 연습 문제 (Practice Problems)

1. `ts_df` 데이터의 'sales' 컬럼에 대해 **14일 이동 표준편차**를 계산하여 `sales_MV_14` 라는 새 컬럼에 저장하고, 상위 20개 행을 출력해보세요. (이동 표준편차는 주가의 변동성을 분석하는 등 데이터의 안정성을 볼 때 사용됩니다.)

In [28]:
# 연습 문제 1번 풀이 공간
ts_df['sales_MV_14'] = ts_df['sales'].rolling(window=14).std()
print(ts_df.head(20))

            sales  sales_MA_7  sales_MA_30  sales_Median_7  sales_MV_14
2023-01-01    152         NaN          NaN             NaN          NaN
2023-01-02    142         NaN          NaN             NaN          NaN
2023-01-03     64         NaN          NaN             NaN          NaN
2023-01-04    156         NaN          NaN             NaN          NaN
2023-01-05    121         NaN          NaN             NaN          NaN
2023-01-06     70         NaN          NaN             NaN          NaN
2023-01-07    152  122.428571          NaN           142.0          NaN
2023-01-08    171  125.142857          NaN           142.0          NaN
2023-01-09    124  122.571429          NaN           124.0          NaN
2023-01-10    137  133.000000          NaN           137.0          NaN
2023-01-11    166  134.428571          NaN           137.0          NaN
2023-01-12    149  138.428571          NaN           149.0          NaN
2023-01-13    153  150.285714          NaN           152.0      

--- 

## 5. 데이터 시프트와 차분

### 💡 개념 (Concept)

> 시계열 분석에서는 현재 시점의 데이터를 과거의 데이터와 비교하는 작업이 매우 흔합니다.
> 
> - **`shift()`**: 데이터의 인덱스는 그대로 둔 채, 값만 특정 기간만큼 뒤로(또는 앞으로) 밀어냅니다. 어제 데이터와 오늘 데이터를 나란히 놓고 비교하거나, 시계열 예측 모델의 피처(feature)를 만들 때 사용됩니다.
> - **`diff()`**: 현재 시점의 데이터에서 바로 이전 시점의 데이터를 뺀 값(차분)을 계산합니다. 데이터의 추세(trend)를 제거하고 정상성(stationarity)을 가진 데이터로 변환하기 위해 사용되며, 변화량 자체를 분석할 때 유용합니다.

In [32]:
# shift() 예제
# 'sales'를 한 칸 뒤로 밀어 'yesterday_sales' 컬럼 생성 - 차분
ts_df['yesterday_sales'] = ts_df['sales'].shift(1)

# diff() 예제
# 어제 대비 오늘의 판매량 변화 계산
ts_df['sales_diff'] = ts_df['sales'].diff(1)

print(ts_df[['sales', 'yesterday_sales', 'sales_diff']].head())

# 일별 판매량 변화량 시각화
fig = px.bar(ts_df, y='sales_diff', title='전일 대비 판매량 변화')
fig.show()

            sales  yesterday_sales  sales_diff
2023-01-01    152              NaN         NaN
2023-01-02    142            152.0       -10.0
2023-01-03     64            142.0       -78.0
2023-01-04    156             64.0        92.0
2023-01-05    121            156.0       -35.0


### ✏️ 연습 문제 (Practice Problems)

1. `ts_df` 데이터에서 **전일 대비 판매량 증감률(%)**을 계산하여 `sales_growth_rate` 컬럼에 저장하고, 상위 5개 행을 출력하세요. 
   (공식: `(오늘 판매량 - 어제 판매량) / 어제 판매량 * 100`)

In [33]:
# 연습 문제 1번 풀이 공간
ts_df['sales_growth_rate'] = (ts_df['sales'].diff(1) / ts_df['sales'].shift(1)) * 100
# 또는
# ts_df['sales_growth_rate'] = ts_df['sales'].pct_change() * 100

print(ts_df[['sales', 'sales_growth_rate']].head())

            sales  sales_growth_rate
2023-01-01    152                NaN
2023-01-02    142          -6.578947
2023-01-03     64         -54.929577
2023-01-04    156         143.750000
2023-01-05    121         -22.435897


In [34]:
# diff, shift 없이 윈도우 함수 rolling 사용
# rolling(2)로 2개의 값을 묶어서 계산
# * ts_df` 데이터에서 **전일 대비 판매량 증감률(%)** 을 계산하여 `sales_growth_rate` 컬럼에 저장하고, 상위 5개 행을 출력하세요. 
#    (공식: `(오늘 판매량 - 어제 판매량) / 어제 판매량 * 100`)

# diff & shift 활용
(ts_df['sales'].diff(1) / ts_df['sales'].shift(1)) * 100

# rolling 함수 활용
ts_df['sales'].rolling(window=2).apply(lambda x: (x.iloc[1] - x.iloc[0]) / x.iloc[0] * 100)

            sales  sales_growth_rate_rolling
2023-01-01    152                        NaN
2023-01-02    142                  -6.578947
2023-01-03     64                 -54.929577
2023-01-04    156                 143.750000
2023-01-05    121                 -22.435897


## 6. 기타 시계열 분석 지원 함수

샘플데이터셋 생성

In [35]:
# 날짜 범위 생성
date_rng = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')

# 날짜 문열 범위 생성
date_str = date_rng.strftime('%m/%d/%Y').tolist()

# 예시 데이터 생성
np.random.seed(42)  # 재현성을 위해 시드 설정
data = np.random.randn(len(date_rng))  # 랜덤 데이터 생성

# 데이터프레임 생성
df = pd.DataFrame([date_rng, date_str], index=['date', 'date_str']).T
df['value'] = data

# 데이터프레임 출력
print(df.head())

                  date    date_str     value
0  2022-01-01 00:00:00  01/01/2022  0.496714
1  2022-01-02 00:00:00  01/02/2022 -0.138264
2  2022-01-03 00:00:00  01/03/2022  0.647689
3  2022-01-04 00:00:00  01/04/2022  1.523030
4  2022-01-05 00:00:00  01/05/2022 -0.234153


### 1. `timedelta_range`
-  일정 간격의 시간 델타 범위를 생성

In [36]:
timedelta_range = pd.timedelta_range(start='0 days', end='2 days', freq='2h')
timedelta_range

TimedeltaIndex(['0 days 00:00:00', '0 days 02:00:00', '0 days 04:00:00',
                '0 days 06:00:00', '0 days 08:00:00', '0 days 10:00:00',
                '0 days 12:00:00', '0 days 14:00:00', '0 days 16:00:00',
                '0 days 18:00:00', '0 days 20:00:00', '0 days 22:00:00',
                '1 days 00:00:00', '1 days 02:00:00', '1 days 04:00:00',
                '1 days 06:00:00', '1 days 08:00:00', '1 days 10:00:00',
                '1 days 12:00:00', '1 days 14:00:00', '1 days 16:00:00',
                '1 days 18:00:00', '1 days 20:00:00', '1 days 22:00:00',
                '2 days 00:00:00'],
               dtype='timedelta64[ns]', freq='2h')

In [37]:
from datetime import datetime
datetime(2025, 6, 9) + timedelta_range

DatetimeIndex(['2025-06-09 00:00:00', '2025-06-09 02:00:00',
               '2025-06-09 04:00:00', '2025-06-09 06:00:00',
               '2025-06-09 08:00:00', '2025-06-09 10:00:00',
               '2025-06-09 12:00:00', '2025-06-09 14:00:00',
               '2025-06-09 16:00:00', '2025-06-09 18:00:00',
               '2025-06-09 20:00:00', '2025-06-09 22:00:00',
               '2025-06-10 00:00:00', '2025-06-10 02:00:00',
               '2025-06-10 04:00:00', '2025-06-10 06:00:00',
               '2025-06-10 08:00:00', '2025-06-10 10:00:00',
               '2025-06-10 12:00:00', '2025-06-10 14:00:00',
               '2025-06-10 16:00:00', '2025-06-10 18:00:00',
               '2025-06-10 20:00:00', '2025-06-10 22:00:00',
               '2025-06-11 00:00:00'],
              dtype='datetime64[ns]', freq='2h')

### 2. `asfreq`
-  특정 주기로 시계열 데이터를 반환

In [38]:
df.set_index('date').asfreq('W').head(10)

,date_str,value
date,,
2022-01-02,01/02/2022,-0.138264
2022-01-09,01/09/2022,-0.469474
2022-01-16,01/16/2022,-0.562288
2022-01-23,01/23/2022,0.067528
2022-01-30,01/30/2022,-0.291694
2022-02-06,02/06/2022,0.208864
2022-02-13,02/13/2022,-0.301104
2022-02-20,02/20/2022,0.324084
2022-02-27,02/27/2022,-0.309212


### 3. `truncate`
-  시계열 데이터의 특정 구간을 잘라오기

In [39]:
sdf = df.set_index('date')
cut_df = sdf.truncate(before='2022-01-05', after='2022-01-10')
cut_df

,date_str,value
date,,
2022-01-05,01/05/2022,-0.234153
2022-01-06,01/06/2022,-0.234137
2022-01-07,01/07/2022,1.579213
2022-01-08,01/08/2022,0.767435
2022-01-09,01/09/2022,-0.469474
2022-01-10,01/10/2022,0.542560


In [43]:
# slice로 데이터 자르기
sdf['2022-01-05':'2022-01-10']

,date_str,value
date,,
2022-01-05,01/05/2022,-0.234153
2022-01-06,01/06/2022,-0.234137
2022-01-07,01/07/2022,1.579213
2022-01-08,01/08/2022,0.767435
2022-01-09,01/09/2022,-0.469474
2022-01-10,01/10/2022,0.542560


### 10. `period_range`
-  특정 기간의 범위를 생성

In [44]:
sales_data = {
    'period': pd.period_range(start='2020-01', end='2020-12', freq='M'),
    'sales': [1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600]
}
sales_df = pd.DataFrame(sales_data)
sales_df.set_index('period', inplace=True)
sales_df

,sales
period,
2020-01,1500
2020-02,1600
2020-03,1700
2020-04,1800
2020-05,1900
2020-06,2000
2020-07,2100
2020-08,2200
2020-09,2300


### 11. `dt` 접근자
-  datetime 속성에 접근할 수 있게 합니다.

In [45]:
sdf = df[['date', 'value']]
sdf['date'] = pd.to_datetime(sdf['date'])
sdf['year'] = sdf['date'].dt.year
sdf['month'] = sdf['date'].dt.month
sdf['day'] = sdf['date'].dt.day
sdf['weekday'] = sdf['date'].dt.weekday
sdf[['date', 'year', 'month', 'day', 'weekday']].head(10)

,date,year,month,day,weekday
0,2022-01-01,2022,1,1,5
1,2022-01-02,2022,1,2,6
2,2022-01-03,2022,1,3,0
3,2022-01-04,2022,1,4,1
4,2022-01-05,2022,1,5,2
5,2022-01-06,2022,1,6,3
6,2022-01-07,2022,1,7,4
7,2022-01-08,2022,1,8,5
8,2022-01-09,2022,1,9,6
9,2022-01-10,2022,1,10,0


### 12. `TimeGrouper`
-  시계열 데이터를 특정 주기로 그룹화

In [46]:
sdf = df[['date', 'value']].copy()
sdf.date = pd.to_datetime(sdf.date)

In [47]:
sdf.groupby(pd.Grouper(key='date', freq='M')).mean()

,value
date,
2022-01-31,-0.201488
2022-02-28,-0.143168
2022-03-31,0.043938
2022-04-30,-0.020252
2022-05-31,-0.083815
2022-06-30,0.291292
2022-07-31,0.098341
2022-08-31,-0.021537
2022-09-30,-0.091070


### 13. `merge_asof`
-  가장 가까운 시계열 데이터로 병합
-  결측치 처리에 유용 - 추천!!

In [48]:
left = pd.DataFrame({
    'date': pd.to_datetime(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04']),
    'value_left': [10, 20, 30, 40]
})

right = pd.DataFrame({
    'date': pd.to_datetime(['2022-01-02', '2022-01-03', '2022-01-05']),
    'value_right': [15, 25, 35]
})

merged = pd.merge_asof(left, right, on='date')
print(merged)

        date  value_left  value_right
0 2022-01-01          10          NaN
1 2022-01-02          20         15.0
2 2022-01-03          30         25.0
3 2022-01-04          40         25.0


In [51]:
# 결측치 처리 시 유용
merged = pd.merge_asof(left, right, on='date', allow_exact_matches=True) # 인근 값으로 채워줌
# merged = pd.merge_asof(left, right, on='date', allow_exact_matches=False) # NaN으로 유지
print(merged)

        date  value_left  value_right
0 2022-01-01          10          NaN
1 2022-01-02          20         15.0
2 2022-01-03          30         25.0
3 2022-01-04          40         25.0


### 14. `expanding`
-  누적 계산

In [52]:
df['expanding_sum'] = df['value'].expanding().sum()
df['expanding_mean'] = df['value'].expanding().mean()
df[['date', 'value', 'expanding_sum', 'expanding_mean']].head(10)

,date,value,expanding_sum,expanding_mean
0,2022-01-01 00:00:00,0.496714,0.496714,0.496714
1,2022-01-02 00:00:00,-0.138264,0.358450,0.179225
2,2022-01-03 00:00:00,0.647689,1.006138,0.335379
3,2022-01-04 00:00:00,1.523030,2.529168,0.632292
4,2022-01-05 00:00:00,-0.234153,2.295015,0.459003
5,2022-01-06 00:00:00,-0.234137,2.060878,0.343480
6,2022-01-07 00:00:00,1.579213,3.640091,0.520013
7,2022-01-08 00:00:00,0.767435,4.407525,0.550941
8,2022-01-09 00:00:00,-0.469474,3.938051,0.437561
9,2022-01-10 00:00:00,0.542560,4.480611,0.448061


### 15. `cumsum`
-  누적 합계를 계산

In [29]:
# cumsum, cummax, cummin, comprod
df['cumsum'] = df['value'].cumsum()
df[['date', 'value', 'cumsum']].head(10)

,date,value,cumsum
0,2022-01-01 00:00:00,0.496714,0.496714
1,2022-01-02 00:00:00,-0.138264,0.358450
2,2022-01-03 00:00:00,0.647689,1.006138
3,2022-01-04 00:00:00,1.523030,2.529168
4,2022-01-05 00:00:00,-0.234153,2.295015
5,2022-01-06 00:00:00,-0.234137,2.060878
6,2022-01-07 00:00:00,1.579213,3.640091
7,2022-01-08 00:00:00,0.767435,4.407525
8,2022-01-09 00:00:00,-0.469474,3.938051
9,2022-01-10 00:00:00,0.542560,4.480611


### 16. `pct_change`
- 이전 값 대비 현재 값의 변화율을 계산합니다.
- 주로 금융 데이터에서 수익률 계산 등에 사용됩니다.


In [53]:
df['pct_change'] = df['value'].pct_change()
df[['date', 'value', 'pct_change']].head(10)

,date,value,pct_change
0,2022-01-01 00:00:00,0.496714,NaN
1,2022-01-02 00:00:00,-0.138264,-1.278358
2,2022-01-03 00:00:00,0.647689,-5.684423
3,2022-01-04 00:00:00,1.523030,1.351485
4,2022-01-05 00:00:00,-0.234153,-1.153742
5,2022-01-06 00:00:00,-0.234137,-0.000070
6,2022-01-07 00:00:00,1.579213,-7.744825
7,2022-01-08 00:00:00,0.767435,-0.514040
8,2022-01-09 00:00:00,-0.469474,-1.611745
9,2022-01-10 00:00:00,0.542560,-2.155675


### 16. `ewm`
 -  지수 가중 이동 평균(Exponential Weighted Moving Average, EWMA) 계산

 -  최근 데이터에 더 큰 가중치를 부여하여 평균을 계산하는 방법

In [54]:
# ewm(options... span / alpha) - 최근 것에 가중치를 줄 지, 오래된 것에 줄 지 설정 가능
df['ewm'] = df['value'].ewm(span=7).mean()
df[['date', 'value', 'ewm']].head(10)

,date,value,ewm
0,2022-01-01 00:00:00,0.496714,0.496714
1,2022-01-02 00:00:00,-0.138264,0.133869
2,2022-01-03 00:00:00,0.647689,0.356061
3,2022-01-04 00:00:00,1.523030,0.782838
4,2022-01-05 00:00:00,-0.234153,0.449484
5,2022-01-06 00:00:00,-0.234137,0.241575
6,2022-01-07 00:00:00,1.579213,0.627499
7,2022-01-08 00:00:00,0.767435,0.666375
8,2022-01-09 00:00:00,-0.469474,0.359361
9,2022-01-10 00:00:00,0.542560,0.407894


## ✏️ 연습 문제 (Practice Problems)

### 문제 1: 데이터 자르기 (`truncate`)
`df` 데이터프레임에서 '2022-03-01'부터 '2022-03-15'까지의 데이터를 잘라내어 `truncated_df`에 저장하고 출력하세요.


In [57]:
# 문제 1 작성공간
truncated_df = df.truncate(before='2022-03-01', after='2022-03-15')
truncated_df

,date,date_str,value,expanding_sum,expanding_mean,pct_change,ewm


### 문제 2: 날짜/시간 속성 추출 (`dt` 접근자)
`sdf` 데이터프레임의 `date` 컬럼에서 요일 이름(예: '월요일', '화요일')을 추출하여 `day_name`이라는 새 컬럼으로 추가하고, 상위 10개 행을 출력하세요.

In [58]:
# 문제 2 풀이
sdf['day_name'] = sdf['date'].dt.day_name()
sdf[['date', 'value', 'day_name']].head(10)

,date,value,day_name
0,2022-01-01,0.496714,Saturday
1,2022-01-02,-0.138264,Sunday
2,2022-01-03,0.647689,Monday
3,2022-01-04,1.523030,Tuesday
4,2022-01-05,-0.234153,Wednesday
5,2022-01-06,-0.234137,Thursday
6,2022-01-07,1.579213,Friday
7,2022-01-08,0.767435,Saturday
8,2022-01-09,-0.469474,Sunday
9,2022-01-10,0.542560,Monday


### 문제 3: 시계열 데이터 그룹화 (`TimeGrouper`)
`sdf` 데이터프레임의 `value` 컬럼을 주(Week) 단위로 그룹화하여 각 주의 평균 `value`를 계산하고 출력하세요.

In [59]:
# 문제 3 풀이
weekly_mean = sdf.groupby(pd.Grouper(key='date', freq='W'))['value'].mean()
weekly_mean

date
2022-01-02    0.179225
2022-01-09    0.511372
2022-01-16   -0.620730
2022-01-23   -0.244502
2022-01-30   -0.503691
2022-02-06   -0.001439
2022-02-13   -0.371130
2022-02-20   -0.385317
2022-02-27    0.051932
2022-03-06   -0.121149
2022-03-13    0.557673
2022-03-20   -0.055603
2022-03-27   -0.314394
2022-04-03    0.227333
2022-04-10   -0.265083
2022-04-17   -0.121722
2022-04-24    0.133696
2022-05-01    0.227369
2022-05-08    0.044576
2022-05-15   -0.484866
2022-05-22    0.223125
2022-05-29   -0.345230
2022-06-05    0.003554
2022-06-12    0.396551
2022-06-19    0.325029
2022-06-26   -0.046343
2022-07-03    0.441305
2022-07-10   -0.273887
2022-07-17   -0.049554
2022-07-24    0.084713
2022-07-31    0.880747
2022-08-07    0.079155
2022-08-14    0.031404
2022-08-21   -0.216787
2022-08-28    0.068317
2022-09-04   -0.220894
2022-09-11    0.618387
2022-09-18   -0.164691
2022-09-25   -0.803323
2022-10-02   -0.016001
2022-10-09    0.074440
2022-10-16    0.178096
2022-10-23   -0.199926
2022-1

### 문제 4: 누적/이동 계산 (`expanding` 또는 `rolling`)
`sdf` 데이터프레임의 `value` 컬럼에 대해 7일 이동 평균(rolling mean)을 계산하여 `rolling_mean_7d` 컬럼으로 추가하고, 상위 10개 행을 출력하세요.

In [60]:
# 문제 4 풀이
sdf['rolling_mean_7d'] = sdf['value'].rolling(window=7).mean()
sdf[['date', 'value', 'rolling_mean_7d']].head(10)

,date,value,rolling_mean_7d
0,2022-01-01,0.496714,NaN
1,2022-01-02,-0.138264,NaN
2,2022-01-03,0.647689,NaN
3,2022-01-04,1.523030,NaN
4,2022-01-05,-0.234153,NaN
5,2022-01-06,-0.234137,NaN
6,2022-01-07,1.579213,0.520013
7,2022-01-08,0.767435,0.558687
8,2022-01-09,-0.469474,0.511372
9,2022-01-10,0.542560,0.496353


### 문제 5: 시계열 데이터 병합 (`merge_asof`)
다음 두 데이터프레임을 `date` 컬럼을 기준으로 `merge_asof`를 사용하여 병합하고 결과를 출력하세요.
`direction='nearest'` 옵션을 사용하여 가장 가까운 날짜로 병합하세요.

In [62]:
df_a = pd.DataFrame({
    'date': pd.to_datetime(['2023-01-01', '2023-01-03', '2023-01-05', '2023-01-07']),
    'event_a': ['A1', 'A2', 'A3', 'A4']
})

df_b = pd.DataFrame({
    'date': pd.to_datetime(['2023-01-02', '2023-01-04', '2023-01-06']),
    'value_b': [100, 200, 300]
})

In [64]:
# 문제 5 풀이
merged_df = pd.merge_asof(df_a, df_b, on='date', direction='nearest')
merged_df

,date,event_a,value_b
0,2023-01-01,A1,100
1,2023-01-03,A2,100
2,2023-01-05,A3,200
3,2023-01-07,A4,300


--- 

## 7. ✏️ 미니 실습 PJT 과제

### 가상 주가 데이터 분석

**시나리오**: 당신은 데이터 분석가입니다. 한 회사의 약 1년간의 주가 데이터를 받아 분석 리포트를 작성해야 합니다. Pandas 시계열 기능을 활용하여 데이터를 분석하고, 주요 인사이트를 시각화하여 보고하세요.

**데이터셋**: 가상의 일별 주가 데이터 (Open, High, Low, Close, Volume)

**분석 목표**: 
1. 월별 평균 종가(Close)를 계산하고 시각화하여 전반적인 주가 추세를 파악합니다.
2. 20일 이동 평균선과 60일 이동 평균선을 계산하고, 원본 종가 데이터와 함께 시각화하여 단기 및 중기 추세를 분석합니다.
3. 일별 수익률(daily return)을 계산하고, 분포를 히스토그램으로 시각화하여 주가의 변동성을 확인합니다.

In [65]:
# 1. 가상 주가 데이터 생성
np.random.seed(101)
dates = pd.date_range('2022-01-01', '2022-12-31', freq='B') # Business day
price_changes = np.random.randn(len(dates)).cumsum()
start_price = 100
close_prices = start_price + price_changes

stock_df = pd.DataFrame({'Close': close_prices}, index=dates)
stock_df['Open'] = stock_df['Close'].shift(1) + np.random.uniform(-2, 2, len(stock_df))
stock_df['High'] = stock_df[['Open', 'Close']].max(axis=1) + np.random.uniform(0, 3, len(stock_df))
stock_df['Low'] = stock_df[['Open', 'Close']].min(axis=1) - np.random.uniform(0, 3, len(stock_df))
stock_df['Volume'] = np.random.randint(100000, 500000, len(stock_df))
stock_df = stock_df.dropna()

print("--- 생성된 주가 데이터 ---")
print(stock_df.head())

--- 생성된 주가 데이터 ---
                 Close        Open        High         Low  Volume
2022-01-04  103.334983  104.605355  107.126032  101.608410  487077
2022-01-05  104.242952  102.254500  106.728008  100.438295  310807
2022-01-06  104.746778  103.148431  106.191822  100.274185  454764
2022-01-07  105.397896  105.397932  108.103716  102.791082  446803
2022-01-10  105.078578  105.309045  106.014115  103.049907  209735


### **분석 과제 1: 월별 평균 종가 계산 및 시각화**

💡 HINT : 월별 평균 종가를 계산하려면 `resample()` 메서드를 사용하여 데이터를 월별로 집계하고, `mean()`을 적용하세요. 시각화는 `plotly.express.line()`을 사용합니다.

In [70]:
import plotly.express as px

In [75]:
stock_df['Close'].head()

2022-01-04    103.334983
2022-01-05    104.242952
2022-01-06    104.746778
2022-01-07    105.397896
2022-01-10    105.078578
Freq: B, Name: Close, dtype: float64

In [72]:
# 월별 평균 종가 계산 (리샘플링)
# 월별 평균 종가 계산
monthly_avg = stock_df['Close'].resample('M').mean()

# DataFrame으로 변환하여 시각화하기 위한 전처리
monthly_avg_df = monthly_avg.reset_index()
monthly_avg_df.columns = ['Date', 'Average Close Price']

# 선 그래프 생성
fig = px.line(monthly_avg_df, 
              x='Date', 
              y='Average Close Price',
              title='Monthly Average Close Price')

# x축 레이블 포맷 설정
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price')

# 그래프 표시
fig.show()

### **분석 과제 2: 이동 평균선 계산 및 시각화**
💡 HINT : 이동 평균선을 계산하려면 `rolling()` 메서드를 사용하여 이동 평균을 계산하고, `mean()`을 적용하세요. 시각화는 `plotly.express.line()`을 사용합니다.


In [73]:
# 20일, 60일 이동 평균 계산 (롤링)
ma20 = stock_df['Close'].rolling(window=20).mean()
ma60 = stock_df['Close'].rolling(window=60).mean()

# DataFrame으로 변환하여 시각화하기 위한 전처리
df_ma = pd.DataFrame({
    'Date': stock_df.index,
    'Close': stock_df['Close'],
    'MA20': ma20,
    'MA60': ma60
})

# 시각화 
fig = px.line(df_ma, 
              x='Date',
              y=['Close', 'MA20', 'MA60'],
              title='Stock Price with Moving Averages')

# x축, y축 레이블 설정
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price')

# 범례 이름 변경
fig.update_layout(
    legend_title='Indicators',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# 그래프 표시
fig.show()

### **분석 과제 3: 일별 수익률 계산 및 시각화**
💡 HINT : 일별 수익률은 `pct_change()` 메서드를 사용하여 계산하고, 시각화는 `plotly.express.histogram()`을 사용합니다.

In [77]:
# 일별 수익률(%) 계산 (diff, shift 또는 pct_change)
daily_returns = stock_df['Close'].pct_change() * 100

# 수익률 분포 시각화
fig = px.histogram(
    daily_returns,
    nbins=50,
    title='Distribution of Daily Returns (%)',
    labels={'value': 'Daily Returns (%)', 'count': 'Frequency'},
)

# 레이아웃 설정
fig.update_layout(
    showlegend=False,
    xaxis_title='Daily Returns (%)',
    yaxis_title='Frequency'
)

# 그래프 표시
fig.show()